<a href="https://colab.research.google.com/github/AliAI11/AliAI11/blob/main/notebooks/load_model_and_profile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch transformers bitsandbytes accelerate rich psutil
print("all dependencies installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.9 MB/s eta 0:00:00
all dependencies installed


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import psutil
from rich.console import Console

console = Console()
console.print("imports were successful")

imports were successful

In [3]:
# loading the model
model_name = "Qwen/Qwen2.5-3B-Instruct"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

console.print(f"Loading {model_name} in 4-bit...")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

console.print("model loaded")

Loading Qwen/Qwen2.5-3B-Instruct in 4-bit...

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model loaded

In [4]:
# checking ram usage
def profile():
    ram = psutil.virtual_memory().used / 1e9
    console.print(f"RAM Used: {ram:.2f} GB")

profile()

RAM Used: 6.06 GB

In [6]:
# testing model
def ask(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=100, temperature=0.3)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

response = ask("What is 6+7?")
console.print(f"Test Answer: {response}")

Test Answer: What is 6+7? 6 + 7 equals 13.